In [24]:
import numpy as np
import pandas as pd

In [43]:
raw_data=pd.read_csv("../data/cleaned_data.csv")
tendecy_one_hot=pd.get_dummies(raw_data.loc[:,"Tendency"])
raw_data=pd.concat([raw_data,tendecy_one_hot],axis=1)
raw_data.drop(['Tendency','DR'],axis=1, inplace=True)


In [65]:
dataset=raw_data.iloc[:,6:]
continuous_attributes=dataset.iloc[:,:20]
categorical_attributes=dataset.iloc[:,20:]
continuous_attributes=(continuous_attributes-continuous_attributes.min())/(continuous_attributes.max()-continuous_attributes.min())
dataset=pd.concat([continuous_attributes,categorical_attributes.iloc[:,1:],categorical_attributes.iloc[:,0]],axis=1)
dataset.to_csv("../data/cleaned_normalized_data.csv",index=False)

In [56]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm

In [96]:
dataset_array=dataset.values
train_x=dataset_array[:1200,:23]
train_y=dataset_array[:1200,23]
test_x=dataset_array[1200:,:23]
test_y=dataset_array[1200:,23]

In [97]:
classifier = svm.SVC(kernel='linear', C = 1.0, verbose=True)
classifier.fit(train_x,train_y)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [98]:
pred_y=classifier.predict(test_x)
i=0
accurate=0
for prediction in pred_y:
    if(prediction==test_y[i]):
        accurate+=1
    i+=1
accuracy= ((accurate/len(test_y))*100)
accuracy
    

77.96976241900649